To Do:
-Cycle through directory to process all zip files
-Process all files in zip file

-Load table of what items are associated with what items, trim table to only items created after 2015
-Determine where data stands
-Calculate cost per packet, divide cost per packet by number of items
-Scope out messy/unmatched data

In [ ]:
import pandas as pd
import numpy
import xlrd
from zipfile import ZipFile

In [ ]:
import numpy
import xlrd
from zipfile import ZipFile

def open_zip(datafile):
    with ZipFile('{0}'.format(datafile), 'r') as myzip:
        myzip.extractall()        


In [ ]:
def process_timesheet(unzipped_datafile, timesheet):
    workbook = xlrd.open_workbook(unzipped_datafile)
    sheet = workbook.sheet_by_index(0)

    #Process first tab of timesheet
    #On the first tab, save cell B3 as the contractor name
    name = sheet.cell_value(2, 1)
    #On the first tab, save cell D18 as the contractor rate
    payrate = sheet.cell_value(17,3)
    if payrate == "Hourly \nRate":
        payrate = sheet.cell_value(18,3)  #likely a more elegant solution, but brute forcing for now
    date = str(int(sheet.cell_value(2,4))) + "-" + str(int(sheet.cell_value(2,5))) + "-" + str(int(sheet.cell_value(2,6)))

    #Process second tab of Excel spreadsheet
    sheet = workbook.sheet_by_index(1)
    #On the second tab, starting with cell A6:E6, save each row until you encounter an empty row
    row = 5
    hours = sheet.cell_value(row,0)
    activity = sheet.cell_value(row, 1)
    description = sheet.cell_value(row, 4)
    while hours != '' and activity != "Total":
        cost = payrate * hours
        timesheet = timesheet.append({'Name':name, 'Payrate':payrate, 'Invoice Date':date, 'Hours':hours, 'Cost': cost, 'Activity':activity, 'Description':description}, ignore_index=True)
        row += 1
        hours = sheet.cell_value(row,0)
        activity = sheet.cell_value(row, 1)
        description = sheet.cell_value(row, 4)
    return timesheet

In [ ]:
import glob

timesheet = pd.DataFrame([],columns=['Name', 'Invoice Date','Payrate','Hours', 'Cost', 'Activity','Description'])

for file in glob.glob("*.zip"):    # only loops over zip files
        open_zip(file)
        
#apply process_timesheet to all unzipped Excel files
for file in glob.glob("*.xlsx"):
        timesheet = process_timesheet(file, timesheet)

In [ ]:
timesheet.groupby('Activity').sum()

In [ ]:
timesheet.groupby('Description').sum()

Processing tasks:
    -Load table of what items are associated with what items, trim table to only items created after 2015
    -determine whether line item is art, item, passage, or packet
    -classify description is useful for calculations
    -Calculate item codes
    -Calculate passage codes
    -Calculate packets


In [ ]:
#replace words that begin with lowercase letter with uppercase letter in Activity
timesheet = timesheet.replace({'item': 'Item', 'writing': 'Writing', 'documentation': 'Documentation', 'batch': 'Batch'}, regex=True)

In [ ]:
timesheet.groupby('Activity').sum()

In [ ]:
item_writing = timesheet.loc[timesheet['Activity']=="Item Writing"]

In [ ]:
print item_writing

In [ ]:
private_label = timesheet.loc[timesheet['Activity']=="Private Label"]

In [ ]:
print private_label

In [ ]:
#Add Summit items to item writing list
summit = private_label[private_label.Description.str.contains('Summit B')]

In [ ]:
print summit

In [ ]:
item_writing = item_writing.append(summit)

In [ ]:
item_review = timesheet.loc[timesheet['Activity']=="Item Review - Internal"]

In [ ]:
print item_review

In [ ]:
item_writing = item_writing.append(item_review)

In [ ]:
print item_writing

In [ ]:
filename = "item writing from timesheets.xlsx"
item_writing.to_excel(filename)

In [ ]:
datafile = "Items with packets.xlsx"
workbook = xlrd.open_workbook(datafile)
sheet = workbook.sheet_by_index(0)

sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
header = ['Internal ID', 'itemcode', 'grade', 'subject', 'packet_information']
packet_info = pd.DataFrame(sheet_data, columns=header)

In [ ]:

print packet_info

In [ ]:
print packet_info

In [ ]:
item_merged = pd.merge(packet_info, item_writing, how='inner', left_on='packet_information', right_on='Description', left_index=False, right_index=False, sort=True)

In [ ]:
print item_merged

In [ ]:
item_merged.groupby('packet_information').count()

    At this point in the analysis, additional transforms were performed in Excel to map the highly variable descriptions to packet names as tracked by the content staff team. 

In [ ]:
datafile = "items with packets post-processing.xlsx"
workbook = xlrd.open_workbook(datafile)
sheet = workbook.sheet_by_index(0)

header = keys = [sheet.cell(0, col_index).value for col_index in xrange(sheet.ncols)]
sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(1,sheet.nrows)]
item_cost = pd.DataFrame(sheet_data, columns=header)

In [ ]:
print item_cost

In [ ]:
for item in item_cost:
    if ['itemArtCount'] != 0.0:
        item_cost['itemCost']=  ['totalPacketCostMinusArt']/['itemCountInPacket'] + ['totalPacketArtCost']/['itemArtCount']
    else: item_cost['itemCost']=  ['totalPacketCostMinusArt']/['itemCountInPacket']